In [1]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [2]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [3]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJFINANCE', 'BERGEPAINT', 'BHARTIARTL',
       'BRITANNIA', 'CANBK', 'CIPLA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DMART', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK',
       'GODREJCP', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE',
       'HEROMOTOCO', 'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI',
       'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD',
       'KOTAKBANK', 'LT', 'LTI', 'M&M', 'MARUTI', 'MCDOWELL-N',
       'MINDTREE', 'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI',
       'PIDILITIND', 'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA',
       'TATACONSUM', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS',
       'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [4]:
mypf['InPortfolio'].value_counts()

0    34
1    26
Name: InPortfolio, dtype: int64

In [5]:
mypf[(mypf['Symbol'] + '.NS') == (stock_ids[1] + '.NS')]['Forecast'].values[0]

946.0

In [6]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import talib 
yfin.pdr_override()

In [7]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  data_source='yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    stock_df['RSI'] = talib.RSI(stock_df['Close Price'], timeperiod=14)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[*********************100%***********************]  1 of 1 completed
DIVISLAB.NS
[*********************100%***********************]  1 of 1 completed
DMART.NS
[*******************

In [8]:
stock_prec_dev('^NSEI').tail(1)

[*********************100%***********************]  1 of 1 completed


,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%
Date,,,,,,,,,,,
2022-10-03,16887.0,17332.0,17271.0,37.732239,17080.0,^NSEI,-1.13,18477.0,15294.0,50.06,49.94


In [9]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [10]:
# portfolio buy/sell
mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-10-03,734.0,752.0,765.0,38.068967,817.0,MOTILALOFS.NS,-10.20,990.0,724.0,3.63,96.37,1000.0,36.24
1,2022-10-03,398.0,424.0,437.0,31.159080,436.0,TATAMOTORS.NS,-8.80,530.0,336.0,31.78,68.22,530.0,33.17
2,2022-10-03,519.0,544.0,552.0,35.594183,583.0,HDFCLIFE.NS,-11.03,733.0,504.0,6.42,93.58,690.0,32.95
3,2022-10-03,69.0,74.0,77.0,36.434363,90.0,NATIONALUM.NS,-23.72,130.0,67.0,2.62,97.38,91.0,31.88
4,2022-10-03,723.0,756.0,746.0,38.545999,730.0,AXISBANK.NS,-0.99,845.0,626.0,44.18,55.82,946.0,30.84
5,2022-10-03,74.0,78.0,78.0,42.762263,96.0,IBREALEST.NS,-22.92,191.0,60.0,10.69,89.31,95.0,28.38
7,2022-10-03,3805.0,4017.0,4096.0,36.623060,4501.0,NAUKRI.NS,-15.47,7020.0,3375.0,11.79,88.21,4867.0,27.91
8,2022-10-03,98.0,102.0,103.0,40.277890,111.0,TATASTEEL.NS,-11.40,141.0,84.0,25.18,74.82,125.0,27.55
9,2022-10-03,2283.0,2358.0,2363.0,38.846971,2398.0,HDFC.NS,-4.79,3001.0,2053.0,24.27,75.73,2905.0,27.24
10,2022-10-03,1394.0,1423.0,1466.0,40.291108,1572.0,INFY.NS,-11.34,1940.0,1365.0,5.00,95.00,1765.0,26.61


In [11]:
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%']*100/df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]['Upside%'].sum()

0     6.625350
1     6.064096
2     6.023876
3     5.828260
4     5.638129
5     5.188395
7     5.102470
8     5.036655
9     4.979981
10    4.864806
11    4.747802
12    4.418728
13    4.281614
14    4.182892
18    3.848334
19    3.608841
20    3.400428
21    3.363864
24    3.336441
25    3.334613
26    3.031134
34    1.888517
36    1.729465
39    1.511911
41    1.458893
59   -3.495494
Name: Upside%, dtype: float64

In [12]:
# prospects buy
df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
6,2022-10-03,510.0,550.0,554.0,22.670542,555.0,ICICIPRULI.NS,-8.13,682.0,447.0,26.77,73.23,654.0,28.24
15,2022-10-03,1006.0,1045.0,1060.0,37.842225,1205.0,TECHM.NS,-16.56,1806.0,965.0,4.82,95.18,1226.0,21.87
16,2022-10-03,1237.0,1272.0,1257.0,39.985124,1194.0,SBILIFE.NS,3.62,1332.0,1025.0,69.14,30.86,1505.0,21.67
17,2022-10-03,6243.0,6382.0,6396.0,40.585715,6533.0,ULTRACEMCO.NS,-4.45,8214.0,5177.0,35.09,64.91,7570.0,21.26
22,2022-10-03,2526.0,2700.0,2744.0,27.092307,2683.0,HEROMOTOCO.NS,-5.87,2928.0,2199.0,44.79,55.21,2988.0,18.29
23,2022-10-03,394.0,403.0,414.0,37.269889,489.0,WIPRO.NS,-19.33,722.0,393.0,0.46,99.54,466.0,18.27
27,2022-10-03,1797.0,1876.0,1888.0,35.622349,1888.0,INDIGO.NS,-4.83,2364.0,1549.0,30.41,69.59,2081.0,15.80
28,2022-10-03,1784.0,1856.0,1854.0,37.443089,1846.0,KOTAKBANK.NS,-3.36,2211.0,1642.0,24.94,75.06,2055.0,15.19
29,2022-10-03,1251.0,1271.0,1240.0,45.771742,1058.0,M&M.NS,18.28,1322.0,691.0,88.80,11.20,1404.0,12.23
30,2022-10-03,557.0,566.0,564.0,44.648513,559.0,DABUR.NS,-0.33,624.0,485.0,51.91,48.09,625.0,12.21


In [13]:
# CCI buy
df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]

,Date,Close Price,20_EMA,50_EMA,RSI,200_EMA,Stock,Dev%_200,Max,Min,Premium%,Discount%,Forecast,Upside%
10,2022-10-03,1394.0,1423.0,1466.0,40.291108,1572.0,INFY.NS,-11.34,1940.0,1365.0,5.00,95.00,1765.0,26.61
15,2022-10-03,1006.0,1045.0,1060.0,37.842225,1205.0,TECHM.NS,-16.56,1806.0,965.0,4.82,95.18,1226.0,21.87
20,2022-10-03,930.0,923.0,938.0,51.986477,1040.0,HCLTECH.NS,-10.55,1352.0,883.0,10.07,89.93,1103.0,18.60
23,2022-10-03,394.0,403.0,414.0,37.269889,489.0,WIPRO.NS,-19.33,722.0,393.0,0.46,99.54,466.0,18.27
26,2022-10-03,2985.0,3065.0,3152.0,35.648016,3379.0,TCS.NS,-11.66,4019.0,2982.0,0.28,99.72,3480.0,16.58
34,2022-10-03,3727.0,3658.0,3680.0,59.418818,4042.0,DIVISLAB.NS,-7.80,5372.0,3449.0,14.45,85.55,4112.0,10.33
39,2022-10-03,3120.0,3194.0,3223.0,43.532403,3517.0,MINDTREE.NS,-11.30,4989.0,2742.0,16.81,83.19,3378.0,8.27
40,2022-10-03,324.0,331.0,319.0,46.339256,280.0,ITC.NS,15.86,346.0,208.0,84.35,15.65,350.0,8.02
42,2022-10-03,605.0,635.0,640.0,29.119798,671.0,BERGEPAINT.NS,-9.77,838.0,561.0,16.05,83.95,651.0,7.60
46,2022-10-03,2622.0,2637.0,2585.0,49.472600,2444.0,HINDUNILVR.NS,7.30,2709.0,1944.0,88.69,11.31,2784.0,6.18
